# Comparison between original model and model with delay

Acapacidade média de atendimento por passo em cada um dos modelos comparados deve ser a mesma. Para isso, o número máxmio de servidores no caso com atraso deve ser igual a $S_d$

$$S_d = t_m \cdot S_o$$

onde $t_m$ é o tempo médio de serviço (1+atraso), $S_o$ é o número de servidores no modelo original sem atrasos.

Para um atraso máximo no serviço de 3 passos ($t_{serM} = 3$), com atrasos distribuidos de maneira uniforme, o $t_m$ seria igual a 3. Para que a comparação seja válida, é necessário que $t_m$ seja um inteiro. Para que isso ocorra, $t_serM$ deve ser ímpar.

$$t_m = \frac{((t_{serM}+1)(t_{serM}+2)/2)-1}{t_{serM}} = \frac{2+3+4}{3} = 3 $$

In [1]:
using JuMP
using HiGHS
using Ipopt
using Juniper
using DelimitedFiles
using Serialization
using Random

include("structs.jl")
include("utils.jl") 
include("optimize_list.jl")

optimize_list (generic function with 1 method)

### Exogenous variables

In [18]:
N = 10           # number of examples
horiz = 20       # total horizon

limit_count = 10 # max number of trials to create a new replacement demand in case of infeasibility

createDemands = false

d_prop = distribution_properties(10, 0, "uniform"); # properties of the demand distribution
a_prop = distribution_properties(1, 0, "uniform");

if createDemands
    d_mat = demand_generator_mat(N, horiz, d_prop.M, d_prop.type, d_prop.std_dev);
    a_mat = demand_generator_mat(N, horiz, a_prop.M, a_prop.type, a_prop.std_dev);
else
    d_fn = string("exogenous_vars//d_mat_", d_prop.type, "_max_", d_prop.M, "_dev_", d_prop.std_dev, ".txt");
    a_fn = string("exogenous_vars//a_mat_", a_prop.type, "_max_", a_prop.M, "_dev_", a_prop.std_dev, ".txt");
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
end
;

### Parametrers

In [19]:
compensateSerDelay = true
compensateCostDelay = false

# bounds
XM = 15            # max queue length 
YM = 20           # max buffer length before dropping calls
phiM = 9          # max adimission to queue
serM_nd = 5       # max number of servers original model
tserM = 3         # max delay in service time

# iniital conditions
X0 = 4
Y0 = 6
L0 = 0
Z0 = 1
tm = (((tserM+1)*(tserM+2)/2)-1)/tserM # average service time

# Non-linear objective function costs
c_blr_nd = 400
c_ser_nd = 0.00000001

# Linear objective function costs
if compensateCostDelay
    c_Z_d = 3/2/tm      # weight of served clients      
    c_ser_d = 1/tm      # weight of server cost
c_L_d = 1/2/tm      # weight of buffer cost
else
    c_Z_d = 3/2               
    c_ser_d = 1         
    c_L_d = 1/2         
end

# Adjust max number of servers for the model with delay
if compensateSerDelay
    serM_d = Int(serM_nd*tm)                  
else
    serM_d = serM_nd                        
end

bds_nd = bounds(XM, YM, phiM, serM_nd, 0)
bds_d = bounds(XM, YM, phiM, serM_d, tserM)

    
ic = initial_conditions(X0, Y0, L0, Z0)

c_nd = c(0, 0, c_ser_nd, c_blr_nd)  # original model costs
c_d = c(c_Z_d, c_L_d, c_ser_d, c_blr_nd)  # model with delay costs

df_input = df_input_generator(horiz, bds_d.serM, bds_d.tserM);

### Model without delay

In [20]:
result_os_nd = optimize_list("nd_os");

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [94]:
# result_om_nd = optimize_list("nd_om");

### Model with delay

In [23]:
include("optimize_list.jl")
result_os_d, d_mat, a_mat = optimize_list("d_os", d_mat, a_mat);

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [83]:
# result_om_d = optimize_list("d_om");

## Results comparison

In [9]:
# results_list = [result_os_nd, result_om_nd, result_os_d, result_om_d]
results_list = [result_os_nd, result_os_d]


res_stats = Dict()

for res in results_list
    result_id = res.id*"_"*string(horiz)*"_"*string(d_prop.M)*"_"*string(a_prop.M)*"_"*string(res.bds.XM)*"_"*string(res.bds.YM)
    cost, av_blr, av_dr, av_ser, total_clients = compute_metrics(res, horiz, N)

    # Print results table
    println(res.id)
    data = hcat(av_blr, total_clients, av_dr, av_ser, cost)
    header = (["average blr", "Average served clients", "Average dropped clients", "Average active Servers", "Average cost function value"]);
    printTable(data, header)

    # add csv line
    add_line_to_csv(result_id, res, horiz, d_prop, a_prop, cost, av_blr, av_dr, av_ser, total_clients)

    # Save results to file
    var_dict = Dict("res"=>res, "a"=>a_mat, "d"=>d_mat)
    fn = string("results//",result_id,".txt")
    serialize(fn, var_dict)
end

nd_os
Results
┌─────────────┬────────────────────────┬─────────────────────────┬────────────────────────┬─────────────────────────────┐
│ average blr │ Average served clients │ Average dropped clients │ Average active Servers │ Average cost function value │
├─────────────┼────────────────────────┼─────────────────────────┼────────────────────────┼─────────────────────────────┤
│       0.215 │                 95.100 │                   0.945 │                  4.705 │                      86.067 │
└─────────────┴────────────────────────┴─────────────────────────┴────────────────────────┴─────────────────────────────┘
d_os
Results
┌─────────────┬────────────────────────┬─────────────────────────┬────────────────────────┬─────────────────────────────┐
│ average blr │ Average served clients │ Average dropped clients │ Average active Servers │ Average cost function value │
├─────────────┼────────────────────────┼─────────────────────────┼────────────────────────┼────────────────────────────

In [17]:
for res in results_list
   plot_results(res, horiz, 6)
end

nd_os


LoadError: BoundsError: attempt to access 10×21 Matrix{Float64} at index [6, 1:50, 1]

### Create demands

In [24]:
createDemands = true

N = 15            # number of examples
horiz = 50        # total horizon
 
M_list = [4, 5, 25]  # max demand
# M_list = [1,3]

std_dev_list = [0]

type_distr = "uniform"
exo_var = "d"

for M in M_list
    if type_distr == "uniform"
        std_dev_list = [0]
    end
    for std in std_dev_list
        d_mat = demand_generator_mat(N, horiz, M, type_distr, std)
        d_mat_fn = "exogenous_vars//"* exo_var*"_mat_"* type_distr* "_max_"* string(M)* "_dev_"* string(std)* ".txt"
        writedlm(d_mat_fn, d_mat)
    end
end


### Check results

In [91]:
function check_results(res, a, d)
    for i in 1:N
        if res.status_opt[i] == 0
            println("The optimization problem n ", i," did not solve to optimality.")
        else
            for t in 1:horiz
                @assert res.X[i,t+1] == res.X[i,t] + res.phi[i,t] - a[t,i] - res.Cin[i,t]
                @assert res.Y[i,t+1] == res.Y[i,t] + res.Q[i,t] - res.phi[i,t]
                @assert res.Z[i,t+1] == res.Z[i,t] + res.Cin[i,t]
                @assert res.L[i,t+1] == res.L[i,t] + res.dr[i,t] + a[t,i]
                @assert res.Cin[i,t] == sum(res.Sl[i,t,:])
            end
        end
    end
    println("All constraints are satisfied")a
end

check_results (generic function with 1 method)